# Projections: Connecting Neural Populations

Projections are `brainpy.state` 's mechanism for connecting neural populations.
They implement the **Communication-Synapse-Output (Comm-Syn-Out)** architecture,
which separates connectivity, synaptic dynamics, and output computation into modular components.

This guide provides a comprehensive understanding of projections in `brainpy.state`.

**Table of Contents**

## Overview

### What are Projections?

A **projection** connects a presynaptic population to a postsynaptic population through:

1. **Communication (Comm)**: How spikes propagate through connections
2. **Synapse (Syn)**: Temporal filtering and synaptic dynamics
3. **Output (Out)**: How synaptic currents affect postsynaptic neurons

**Key benefits:**

- Modular design (swap components independently)
- Biologically realistic (separate connectivity and dynamics)
- Efficient (optimized sparse operations)
- Flexible (combine components in different ways)

### The Comm-Syn-Out Architecture

In [ ]:
Presynaptic       Communication        Synapse          Output        Postsynaptic
Population    ──►  (Connectivity)  ──►  (Dynamics)  ──►  (Current) ──►  Population

Spikes        ──►  Weight matrix   ──►  g(t)        ──►  I_syn     ──►  Neurons
                   Sparse/Dense         Expon/Alpha     CUBA/COBA

**Flow:**

1. Presynaptic spikes arrive
2. Communication: Spikes propagate through connectivity matrix
3. Synapse: Temporal dynamics filter the signal
4. Output: Convert to current/conductance
5. Postsynaptic neurons receive input

### Types of Projections

BrainPy provides two main projection types:

**AlignPostProj**
   - Align synaptic states with postsynaptic neurons
   - Most common for standard neural networks
   - Efficient memory layout

**AlignPreProj**
   - Align synaptic states with presynaptic neurons
   - Useful for certain learning rules
   - Different memory organization

For most use cases, use `AlignPostProj`.

## Communication Layer

The Communication layer defines **how spikes propagate** through connections.

### Dense Connectivity

All neurons potentially connected (though weights may be zero).

**Use case:** Small networks, fully connected layers

In [232]:
import brainpy
import brainstate
import brainunit as u
import braintools

# Dense linear transformation
comm = brainstate.nn.Linear(
    100,                    # in_size
    50,                     # out_size
    w_init=braintools.init.KaimingNormal(),
    b_init=None             # No bias for synapses
)

**Characteristics:**

- Memory: O(n_pre × n_post)
- Computation: Full matrix multiplication
- Best for: Small networks, fully connected architectures

### Sparse Connectivity

Only a subset of connections exist (biologically realistic).

**Use case:** Large networks, biological connectivity patterns

#### Event-Based Fixed Probability

Connect neurons with fixed probability.

In [233]:
# Sparse random connectivity (2% connection probability)
comm = brainstate.nn.EventFixedProb(
    1000,                   # pre_size
    800,                    # post_size
    conn_num=0.02,          # 2% connectivity
    conn_weight=0.5         # Synaptic weight (unitless for event-based)
)

**Characteristics:**

- Memory: O(n_pre × n_post × prob)
- Computation: Only active connections
- Best for: Large-scale networks, biological models

#### Event-Based All-to-All

All neurons connected (but stored sparsely).

In [234]:
# All-to-all sparse (event-driven)
comm = brainstate.nn.AllToAll(
    100,                    # pre_size
    100,                    # post_size
    0.3              # Unitless weight
)

#### Event-Based One-to-One

One-to-one mapping (same size populations).

In [235]:
size=100
weight=1.0

# One-to-one connections
comm = brainstate.nn.OneToOne(
    size,
    weight  # Unitless weight
)

**Use case:** Feedforward pathways, identity mappings

### Comparison Table


   * - Type
     - Memory
     - Speed
     - Use Case
     - Example
   * - Linear (Dense)
     - High (O(n²))
     - Fast (optimized)
     - Small networks
     - Fully connected
   * - EventFixedProb
     - Low (O(n²p))
     - Very fast
     - Large networks
     - Cortical connectivity
   * - EventAll2All
     - Medium
     - Fast
     - Medium networks
     - Recurrent layers
   * - EventOne2One
     - Minimal (O(n))
     - Fastest
     - Feedforward
     - Sensory pathways

## Synapse Layer

The Synapse layer defines **temporal dynamics** of synaptic transmission.

### Exponential Synapse

Single exponential decay (most common).

**Dynamics:**


$$
\tau \frac{dg}{dt} = -g + \sum_k \delta(t - t_k)
$$
**Implementation:**

In [236]:
# Exponential synapse with 5ms time constant
syn = brainpy.state.Expon.desc(
    size=100,           # Postsynaptic population size
    tau=5.0 * u.ms      # Decay time constant
)

**Characteristics:**

- Single time constant
- Fast computation
- Good for most applications

**When to use:** Default choice for most models

### Alpha Synapse

Dual exponential with rise and decay.

**Dynamics:**


$$
\tau \frac{dg}{dt} = -g + h
\tau \frac{dh}{dt} = -h + \sum_k \delta(t - t_k)
$$
**Implementation:**

In [237]:
# Alpha synapse
syn = brainpy.state.Alpha.desc(
    size=100,
    tau=10.0 * u.ms     # Characteristic time
)

**Characteristics:**

- Realistic rise time
- Smoother response
- Slightly slower computation

**When to use:** When rise time matters, more biological realism

### NMDA Synapse

Voltage-dependent NMDA receptors.

**Dynamics:**


$$
g_{NMDA} = \frac{g}{1 + \eta [Mg^{2+}] e^{-\gamma V}}
$$
**Implementation:**

In [238]:
# NMDA receptor
syn = brainpy.state.BioNMDA(
    in_size=100,
    T_dur=100.0 * u.ms,    # Slow decay
    T=2.0 * u.ms,       # Fast rise
    alpha1=0.5 / u.mM,              # Mg²⁺ sensitivity
    g_initializer=1.2 * u.mM           # Mg²⁺ concentration
)

**Characteristics:**

- Voltage-dependent
- Slow kinetics
- Important for plasticity

**When to use:** Long-term potentiation, working memory models

### AMPA Synapse

Fast glutamatergic transmission.

In [239]:
# AMPA receptor (fast excitation)
syn = brainpy.state.AMPA.desc(
    size=100,
    tau=2.0 * u.ms      # Fast decay (~2ms)
)

**When to use:** Fast excitatory transmission

### GABA Synapse

Inhibitory transmission.

**GABAa (fast):**

In [240]:
# GABAa receptor (fast inhibition)
syn = brainpy.state.GABAa.desc(
    size=100,
    tau=6.0 * u.ms      # ~6ms decay
)

**GABAb (slow):**

In [241]:
# GABAb receptor (slow inhibition)
syn = brainpy.state.GABAa(
    in_size=100,
    T_dur=150.0 * u.ms,    # Very slow
    T=3.5 * u.ms
)

**When to use:**
- GABAa: Fast inhibition, cortical networks
- GABAb: Slow inhibition, rhythm generation

### Custom Synapses

Create custom synaptic dynamics by subclassing `Synapse`.

In [242]:
import jax.numpy as jnp

class DoubleExpSynapse(brainpy.state.Synapse):
    """Custom synapse with two time constants."""

    def __init__(self, size, tau_fast=2*u.ms, tau_slow=10*u.ms, **kwargs):
        super().__init__(size, **kwargs)
        self.tau_fast = tau_fast
        self.tau_slow = tau_slow

        # State variables
        self.g_fast = brainstate.ShortTermState(jnp.zeros(size))
        self.g_slow = brainstate.ShortTermState(jnp.zeros(size))

    def reset_state(self, batch_size=None):
        shape = self.size if batch_size is None else (batch_size, self.size)
        self.g_fast.value = jnp.zeros(shape)
        self.g_slow.value = jnp.zeros(shape)

    def update(self, x):
        dt = brainstate.environ.get_dt()

        # Fast component
        dg_fast = -self.g_fast.value / self.tau_fast.to_decimal(u.ms)
        self.g_fast.value += dg_fast * dt.to_decimal(u.ms) + x * 0.7

        # Slow component
        dg_slow = -self.g_slow.value / self.tau_slow.to_decimal(u.ms)
        self.g_slow.value += dg_slow * dt.to_decimal(u.ms) + x * 0.3

        return self.g_fast.value + self.g_slow.value

## Output Layer

The Output layer defines **how synaptic conductance affects neurons**.

### CUBA (Current-Based)

Synaptic conductance directly becomes current.

**Model:**


$$
I_{syn} = g_{syn}
$$
**Implementation:**

In [243]:
# Define population sizes
pre_size = 100
post_size = 50

# Define connectivity parameters
conn_num = 0.1
conn_weight = 0.5

comm = brainstate.nn.EventFixedProb(
    pre_size, post_size, conn_num, conn_weight
)

**Characteristics:**

- Simple and fast
- No voltage dependence
- Good for rate-based models

**When to use:**
- Abstract models
- When voltage dependence not important
- Faster computation needed

### COBA (Conductance-Based)

Synaptic conductance with reversal potential.

**Model:**


$$
I_{syn} = g_{syn} (E_{syn} - V_{post})
$$
**Implementation:**

In [244]:
# Excitatory conductance-based
out_exc = brainpy.state.COBA.desc(E=0.0 * u.mV)

# Inhibitory conductance-based
out_inh = brainpy.state.COBA.desc(E=-80.0 * u.mV)

**Characteristics:**

- Voltage-dependent
- Biologically realistic
- Self-limiting (saturates near reversal)

**When to use:**
- Biologically detailed models
- When voltage dependence matters
- Shunting inhibition needed

### MgBlock (NMDA)

Voltage-dependent magnesium block for NMDA.

In [245]:
# NMDA with Mg²⁺ block
out_nmda = brainpy.state.MgBlock.desc(
    E=0.0 * u.mV,
    cc_Mg=1.2 * u.mM,
    alpha=0.062 / u.mV,
    beta=3.57
)

**When to use:** NMDA receptors, voltage-dependent plasticity

## Complete Projection Examples

### Example 1: Simple Feedforward

In [246]:
import brainpy as bp
import brainstate
import brainunit as u
import jax.numpy as jnp

# Create populations
pre = brainpy.state.LIF(100, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)
post = brainpy.state.LIF(50, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)

# Create projection: 100 → 50 neurons
proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(
        100,                   # pre_size
        50,                    # post_size
        conn_num=0.1,          # 10% connectivity
        conn_weight=0.5        # Weight
    ),
    syn=brainpy.state.Expon.desc(
        in_size=50,               # Postsynaptic size
        tau=5.0 * u.ms
    ),
    out=brainpy.state.CUBA.desc(),
    post=post                  # Postsynaptic population
)

# Initialize
brainstate.nn.init_all_states([pre, post, proj])

# Simulate
def step(t, i, inp):
    with brainstate.environ.context(t=t, i=i):
        # Get presynaptic spikes
        pre_spikes = pre.get_spike()

        # Update projection
        proj(pre_spikes)

        # Update neurons
        pre(inp)
        post(0.0 * u.nA)  # Projection provides input

        return pre.get_spike(), post.get_spike()

### Example 2: Excitatory-Inhibitory Network

In [247]:
class EINetwork(brainstate.nn.Module):
    def __init__(self, n_exc=800, n_inh=200):
        super().__init__()

        # Populations
        self.E = brainpy.state.LIF(n_exc, V_rest=-65*u.mV, V_th=-50*u.mV, tau=15*u.ms)
        self.I = brainpy.state.LIF(n_inh, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)

        # E → E projection (AMPA, excitatory)
        self.E2E = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(n_exc, n_exc, conn_num=0.02, conn_weight=0.6*u.mS),
            syn=brainpy.state.AMPA.desc(n_exc, tau=2.0*u.ms),
            out=brainpy.state.COBA.desc(E=0.0*u.mV),
            post=self.E
        )

        # E → I projection (AMPA, excitatory)
        self.E2I = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(n_exc, n_inh, conn_num=0.02, conn_weight=0.6*u.mS),
            syn=brainpy.state.AMPA.desc(n_inh, tau=2.0*u.ms),
            out=brainpy.state.COBA.desc(E=0.0*u.mV),
            post=self.I
        )

        # I → E projection (GABAa, inhibitory)
        self.I2E = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(n_inh, n_exc, conn_num=0.02, conn_weight=6.7*u.mS),
            syn=brainpy.state.GABAa.desc(n_exc, tau=6.0*u.ms),
            out=brainpy.state.COBA.desc(E=-80.0*u.mV),
            post=self.E
        )

        # I → I projection (GABAa, inhibitory)
        self.I2I = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(n_inh, n_inh, conn_num=0.02, conn_weight=6.7*u.mS),
            syn=brainpy.state.GABAa.desc(n_inh, tau=6.0*u.ms),
            out=brainpy.state.COBA.desc(E=-80.0*u.mV),
            post=self.I
        )

    def update(self, t, i, inp_e, inp_i):
        with brainstate.environ.context(t=t, i=i):
            # Get spikes BEFORE updating neurons
            spk_e = self.E.get_spike()
            spk_i = self.I.get_spike()

            # Update all projections
            self.E2E(spk_e)
            self.E2I(spk_e)
            self.I2E(spk_i)
            self.I2I(spk_i)

            # Update neurons (projections provide synaptic input)
            self.E(inp_e)
            self.I(inp_i)

            return spk_e, spk_i

### Example 3: Multi-Timescale Synapses

Combine AMPA (fast) and NMDA (slow) for realistic excitation.

In [248]:
class DualExcitatory(brainstate.nn.Module):
    """E → E with both AMPA and NMDA."""

    def __init__(self, n_pre=100, n_post=100):
        super().__init__()

        self.post = brainpy.state.LIF(n_post, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)

        # Fast AMPA component
        self.ampa_proj = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(n_pre, n_post, conn_num=0.1, conn_weight=0.3*u.mS),
            syn=brainpy.state.AMPA.desc(n_post, tau=2.0*u.ms),
            out=brainpy.state.COBA.desc(E=0.0*u.mV),
            post=self.post
        )

        # Slow NMDA component
        self.nmda_proj = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(n_pre, n_post, conn_num=0.1, conn_weight=0.3*u.mS),
            syn=brainpy.state.NMDA.desc(n_post, tau_decay=100.0*u.ms, tau_rise=2.0*u.ms),
            out=brainpy.state.MgBlock.desc(E=0.0*u.mV, cc_Mg=1.2*u.mM),
            post=self.post
        )

    def update(self, t, i, pre_spikes):
        with brainstate.environ.context(t=t, i=i):
            # Both projections share same presynaptic spikes
            self.ampa_proj(pre_spikes)
            self.nmda_proj(pre_spikes)

            # Post receives combined input
            self.post(0.0 * u.nA)

            return self.post.get_spike()

## Advanced Topics

### Delay Projections

Add synaptic delays to projections.

In [249]:
import jax

# Define post_neurons for demonstration
post_neurons = brainpy.state.LIF(100, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)

# To implement delay, use a separate Delay module
delay_time = 5.0 * u.ms

# Create a network with delay
class DelayedProjection(brainstate.nn.Module):
    def __init__(self, pre_size, post_size):
        super().__init__()
        
        # Delay buffer for spikes
        self.delay = brainstate.nn.Delay(
            jax.ShapeDtypeStruct((pre_size,), bool), 
            delay_time
        )
        
        # Standard projection
        self.proj = brainpy.state.AlignPostProj(
            comm=brainstate.nn.EventFixedProb(pre_size, post_size, conn_num=0.1, conn_weight=0.5),
            syn=brainpy.state.Expon.desc(post_size, tau=5.0*u.ms),
            out=brainpy.state.CUBA.desc(),
            post=post_neurons
        )
    
    def update(self, pre_spikes):
        # Retrieve delayed spikes
        delayed_spikes = self.delay.retrieve_at_step(
            u.math.asarray(delay_time / brainstate.environ.get_dt(), dtype=int)
        )
        # Update projection with delayed spikes
        self.proj(delayed_spikes)
        # Store current spikes in delay buffer
        self.delay(pre_spikes)

# Example usage:
# delayed_proj = DelayedProjection(100, 100)

**Use cases:**
- Biologically realistic transmission delays
- Axonal conduction delays
- Synchronization studies

### Heterogeneous Weights

Different weights for different connections.

In [250]:
import jax.numpy as jnp

# Custom weight matrix
n_pre, n_post = 100, 50
weights = jnp.abs(brainstate.random.randn(n_pre, n_post)) * 0.5

# Note: EventJitFPHomoLinear may not support heterogeneous weights
# For custom weights, consider using Linear or custom communication layer
# comm = brainstate.nn.Linear(n_pre, n_post, w_init=lambda key, shape: weights)

### Learning Synapses

Combine with plasticity (see ../tutorials/advanced/06-synaptic-plasticity).

In [251]:
# Projection with learnable weights
class PlasticProjection(brainstate.nn.Module):
    def __init__(self, n_pre, n_post):
        super().__init__()

        # Initialize weights as parameters
        self.weights = brainstate.ParamState(
            jnp.ones((n_pre, n_post)) * 0.5 * u.mS
        )

        self.proj = brainpy.state.AlignPostProj(
            comm=CustomComm(self.weights),  # Use learnable weights
            syn=brainpy.state.Expon.desc(n_post, tau=5.0*u.ms),
            out=brainpy.state.CUBA.desc(),
            post=post_neurons
        )

    def update_weights(self, dw):
        """Update weights based on learning rule."""
        self.weights.value += dw

## Best Practices

### Choosing Communication Type

**Use EventFixedProb when:**
- Large networks (>1000 neurons)
- Sparse connectivity (<10%)
- Biological models

**Use Linear when:**
- Small networks (<1000 neurons)
- Fully connected layers
- Training with gradients

**Use EventOne2One when:**
- Same-size populations
- Feedforward pathways
- Identity mappings

### Choosing Synapse Type

**Use Expon when:**
- Default choice for most models
- Fast computation needed
- Simple dynamics sufficient

**Use Alpha when:**
- Rise time is important
- More biological realism
- Smoother responses

**Use AMPA/NMDA/GABA when:**
- Specific receptor types matter
- Pharmacological studies
- Detailed biological models

### Choosing Output Type

**Use CUBA when:**
- Abstract models
- Training with gradients
- Speed is critical

**Use COBA when:**
- Biological realism needed
- Voltage dependence matters
- Shunting inhibition required

### Performance Tips

1. **Sparse over Dense:** Use sparse connectivity for large networks
2. **Batch initialization:** Initialize all modules together
3. **JIT compile:** Wrap simulation loop with `@brainstate.transform.jit`
4. **Appropriate precision:** Use float32 unless high precision needed
5. **Minimize communication:** Group projections with same connectivity

### Common Patterns

**Pattern 1: Dale's Principle**

Neurons are either excitatory OR inhibitory (not both).

In [252]:
# Set simulation timestep if not already set
brainstate.environ.set(dt=0.1 * u.ms)

# Separate excitatory and inhibitory populations
E = brainpy.state.LIF(800, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)
I = brainpy.state.LIF(200, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)

# Initialize states
brainstate.nn.init_all_states([E, I])

# E always excitatory (E=0mV)
# I always inhibitory (E=-80mV)

[LIF(
   in_size=(800,),
   out_size=(800,),
   spk_reset=soft,
   spk_fun=ReluGrad(alpha=0.3, width=1.0),
   R=1. * ohm,
   tau=10 * msecond,
   V_th=-50 * mvolt,
   V_rest=-65 * mvolt,
   V_reset=0. * mvolt,
   V_initializer=Constant(value=0.0 * mvolt),
   V=HiddenState(
     value=~float32[800] * mvolt
   )
 ),
 LIF(
   in_size=(200,),
   out_size=(200,),
   spk_reset=soft,
   spk_fun=ReluGrad(alpha=0.3, width=1.0),
   R=1. * ohm,
   tau=10 * msecond,
   V_th=-50 * mvolt,
   V_rest=-65 * mvolt,
   V_reset=0. * mvolt,
   V_initializer=Constant(value=0.0 * mvolt),
   V=HiddenState(
     value=~float32[200] * mvolt
   )
 )]

**Pattern 2: Balanced Networks**

Excitation balanced by inhibition.

In [253]:
# Strong inhibition to balance excitation
w_exc = 0.6 * u.mS
w_inh = 6.7 * u.mS  # ~10× stronger

# More E neurons than I (4:1 ratio)
n_exc = 800
n_inh = 200

**Pattern 3: Recurrent Loops**

Self-connections for persistent activity.

In [254]:
# Set simulation timestep if not already set
brainstate.environ.set(dt=0.1 * u.ms)

# Define E population for demonstration
E = brainpy.state.LIF(800, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)
n_exc = 800

# Initialize states
brainstate.nn.init_all_states(E)

# Excitatory recurrence (working memory)
E2E = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(n_exc, n_exc, conn_num=0.02, conn_weight=0.5*u.mS),
    syn=brainpy.state.Expon.desc(n_exc, tau=5*u.ms),
    out=brainpy.state.COBA.desc(E=0*u.mV),
    post=E
)

# Initialize projection states
brainstate.nn.init_all_states(E2E)

AlignPostProj(
  name=AlignPostProj30,
  modules=(),
  merging=True,
  comm=EventFixedNumConn(
    in_size=(800,),
    out_size=(800,),
    efferent_target=post,
    conn_num=16,
    seed=None,
    allow_multi_conn=True,
    weight=ParamState(
      value=~float32[] * msiemens
    ),
    conn=FixedPostNumConn(float32[800, 800], nse=12800)
  ),
  syn=Expon(
    in_size=(800,),
    out_size=(800,),
    tau=5 * msecond,
    g_initializer=Constant(value=0.0 * msiemens),
    g=HiddenState(
      value=~float32[800] * msiemens
    )
  ),
  out=COBA(
    E=0 * mvolt
  ),
  post=LIF(
    in_size=(800,),
    out_size=(800,),
    before_updates={
      "(<class 'brainpy.state.Expon'>, (800,), {'tau': 5 * msecond}) // (<class 'brainpy.state.COBA'>, (), {'E': 0 * mvolt})": _AlignPost(
        syn=Expon(...),
        out=COBA(...)
      )
    },
    current_inputs={
      'AlignPostProj30': COBA(...)
    },
    spk_reset=soft,
    spk_fun=ReluGrad(alpha=0.3, width=1.0),
    R=1. * ohm,
    tau=10 *

## Troubleshooting

### Issue: Spikes not propagating

**Symptoms:** Postsynaptic neurons don't receive input

**Solutions:**

1. Check spike timing: Call `get_spike()` BEFORE updating
2. Verify connectivity: Check `prob` and `weight`
3. Check update order: Projections before neurons

In [255]:
# Set simulation timestep
brainstate.environ.set(dt=0.1 * u.ms)

# Define neurons and projection for demonstration
pre = brainpy.state.LIF(100, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)
post = brainpy.state.LIF(50, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)
proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(100, 50, conn_num=0.1, conn_weight=0.5),
    syn=brainpy.state.Expon.desc(50, tau=5.0*u.ms),
    out=brainpy.state.CUBA.desc(),
    post=post
)

# Initialize all states
brainstate.nn.init_all_states([pre, post, proj])

# Define input current
inp = jnp.ones(100) * 5.0 * u.nA

# CORRECT order - in update function context
def correct_update(t, i):
    with brainstate.environ.context(t=t, i=i):
        spk = pre.get_spike()  # Get spikes from previous step
        proj(spk)               # Update projection
        pre(inp)                # Update neurons
        return spk

# Example: run one step
result = correct_update(0.0*u.ms, 0)


### Issue: Network silent or exploding

**Symptoms:** No activity or runaway firing

**Solutions:**

1. Balance E/I weights (I should be ~10× stronger)
2. Check reversal potentials (E=0mV, I=-80mV)
3. Verify threshold and reset values
4. Add external input

In [256]:
# Balanced weights
w_exc = 0.5 * u.mS
w_inh = 5.0 * u.mS  # Strong inhibition

# Proper reversal potentials
out_exc = brainpy.state.COBA.desc(E=0.0 * u.mV)
out_inh = brainpy.state.COBA.desc(E=-80.0 * u.mV)

### Issue: Slow simulation

**Solutions:**

1. Use sparse connectivity (EventFixedProb)
2. Use JIT compilation
3. Use CUBA instead of COBA (if appropriate)
4. Reduce connectivity or neurons

In [257]:
# Fast configuration
@brainstate.transform.jit
def simulate_step(net, t, i, inp):
    with brainstate.environ.context(t=t, i=i):
        return net.update(t, i, inp, inp)

# Sparse connectivity
comm = brainstate.nn.EventFixedProb(1000, 1000, conn_num=0.02, conn_weight=0.5)

## Further Reading

- ../tutorials/basic/03-network-connections - Network connections tutorial
- architecture - Overall BrainPy architecture
- synapses - Detailed synapse models
- ../tutorials/advanced/06-synaptic-plasticity - Learning in projections
- ../tutorials/advanced/07-large-scale-simulations - Scaling projections

## Summary

**Key takeaways:**

✅ Projections use Comm-Syn-Out architecture

✅ Communication: Dense (Linear) or Sparse (EventFixedProb)

✅ Synapse: Temporal dynamics (Expon, Alpha, AMPA, GABA, NMDA)

✅ Output: Current-based (CUBA) or Conductance-based (COBA)

✅ Choose components based on scale, realism, and performance needs

✅ Follow Dale's principle and balanced E/I patterns

✅ Get spikes BEFORE updating for correct propagation

**Quick reference:**

In [258]:
# Define postsynaptic neurons for template
post_neurons = brainpy.state.LIF(50, V_rest=-65*u.mV, V_th=-50*u.mV, tau=10*u.ms)
n_pre = 100
n_post = 50

# Standard projection template
proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(n_pre, n_post, conn_num=0.1, conn_weight=0.5*u.mS),
    syn=brainpy.state.Expon.desc(n_post, tau=5.0*u.ms),
    out=brainpy.state.COBA.desc(E=0.0*u.mV),
    post=post_neurons
)

# Usage in network
# def update(self, t, i):
#     with brainstate.environ.context(t=t, i=i):
#         spk = self.pre.get_spike()  # Get spikes first
#         self.proj(spk)               # Update projection
#         self.pre(inp)                # Update neurons
#         self.post(0*u.nA)